In [1]:
import os
import sys
import time
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path
from datetime import datetime
from model_settings import ms
from df_collector import df_collector
root = Path().resolve().parent.parent
df_collector.root = root
options = df_collector.cboe_spx_barriers()
options

E:\Python\Lib\site-packages\df_collector\df_collector.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(dfs,ignore_index=True)


,Unnamed: 0,spot_price,strike_price,barrier,days_to_maturity,updown,outin,w,barrier_type_name,rebate,dividend_rate,risk_free_rate,theta,kappa,rho,eta,v0,calculation_date,date,barrier_price
0,0.0,4441.75,3997.575,2220.875,60.0,Down,Out,call,DownOut,0.0,0.0,0.04,0.056729,20.262203,-0.726127,3.896447,0.068531,2022-02-16 14:03:38.885,2022-02-16,515.613589
1,1.0,4441.75,3997.575,2220.875,60.0,Down,Out,put,DownOut,0.0,0.0,0.04,0.056729,20.262203,-0.726127,3.896447,0.068531,2022-02-16 14:03:38.885,2022-02-16,37.861034
2,2.0,4441.75,3997.575,2220.875,60.0,Down,In,call,DownIn,0.0,0.0,0.04,0.056729,20.262203,-0.726127,3.896447,0.068531,2022-02-16 14:03:38.885,2022-02-16,0.000000
3,3.0,4441.75,3997.575,2220.875,60.0,Down,In,put,DownIn,0.0,0.0,0.04,0.056729,20.262203,-0.726127,3.896447,0.068531,2022-02-16 14:03:38.885,2022-02-16,7.031708
4,4.0,4441.75,3997.575,2220.875,90.0,Down,Out,call,DownOut,0.0,0.0,0.04,0.056729,20.262203,-0.726127,3.896447,0.068531,2022-02-16 14:03:38.885,2022-02-16,546.301779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27064795,NaN,5813.50,6394.850,8720.250,540.0,Up,In,put,UpIn,0.0,0.0,0.04,0.087499,1.982234,-0.657095,1.705566,0.018946,2024-10-15 16:50:04.131,2024-10-15,6.577341
27064796,NaN,5813.50,6394.850,8720.250,720.0,Up,Out,call,UpOut,0.0,0.0,0.04,0.087499,1.982234,-0.657095,1.705566,0.018946,2024-10-15 16:50:04.131,2024-10-15,377.391470
27064797,NaN,5813.50,6394.850,8720.250,720.0,Up,Out,put,UpOut,0.0,0.0,0.04,0.087499,1.982234,-0.657095,1.705566,0.018946,2024-10-15 16:50:04.131,2024-10-15,650.459798
27064798,NaN,5813.50,6394.850,8720.250,720.0,Up,In,call,UpIn,0.0,0.0,0.04,0.087499,1.982234,-0.657095,1.705566,0.018946,2024-10-15 16:50:04.131,2024-10-15,189.459253


In [2]:
dataset = options.copy().iloc[:,1:].dropna().reset_index(drop=True)
dataset

,spot_price,strike_price,barrier,days_to_maturity,updown,outin,w,barrier_type_name,rebate,dividend_rate,risk_free_rate,theta,kappa,rho,eta,v0,calculation_date,date,barrier_price
0,4441.75,3997.575,2220.875,60.0,Down,Out,call,DownOut,0.0,0.0,0.04,0.056729,20.262203,-0.726127,3.896447,0.068531,2022-02-16 14:03:38.885,2022-02-16,515.613589
1,4441.75,3997.575,2220.875,60.0,Down,Out,put,DownOut,0.0,0.0,0.04,0.056729,20.262203,-0.726127,3.896447,0.068531,2022-02-16 14:03:38.885,2022-02-16,37.861034
2,4441.75,3997.575,2220.875,60.0,Down,In,call,DownIn,0.0,0.0,0.04,0.056729,20.262203,-0.726127,3.896447,0.068531,2022-02-16 14:03:38.885,2022-02-16,0.000000
3,4441.75,3997.575,2220.875,60.0,Down,In,put,DownIn,0.0,0.0,0.04,0.056729,20.262203,-0.726127,3.896447,0.068531,2022-02-16 14:03:38.885,2022-02-16,7.031708
4,4441.75,3997.575,2220.875,90.0,Down,Out,call,DownOut,0.0,0.0,0.04,0.056729,20.262203,-0.726127,3.896447,0.068531,2022-02-16 14:03:38.885,2022-02-16,546.301779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26948695,5813.50,6394.850,8720.250,540.0,Up,In,put,UpIn,0.0,0.0,0.04,0.087499,1.982234,-0.657095,1.705566,0.018946,2024-10-15 16:50:04.131,2024-10-15,6.577341
26948696,5813.50,6394.850,8720.250,720.0,Up,Out,call,UpOut,0.0,0.0,0.04,0.087499,1.982234,-0.657095,1.705566,0.018946,2024-10-15 16:50:04.131,2024-10-15,377.391470
26948697,5813.50,6394.850,8720.250,720.0,Up,Out,put,UpOut,0.0,0.0,0.04,0.087499,1.982234,-0.657095,1.705566,0.018946,2024-10-15 16:50:04.131,2024-10-15,650.459798
26948698,5813.50,6394.850,8720.250,720.0,Up,In,call,UpIn,0.0,0.0,0.04,0.087499,1.982234,-0.657095,1.705566,0.018946,2024-10-15 16:50:04.131,2024-10-15,189.459253


In [3]:
price_name = 'barrier_price' 
dataset = dataset[dataset['barrier_price']<dataset['spot_price']].dropna().reset_index(drop=True).copy()

In [4]:
dataset['observed_price'] = np.maximum(dataset[price_name] + np.random.normal(scale=(0.15)**2,size=dataset.shape[0]),0)
dataset['moneyness'] = ms.df_moneyness(dataset)
dataset['relative_moneyness'] = dataset['moneyness']/dataset['spot_price']
dataset['relative_barrier'] = dataset['barrier']/dataset['spot_price']
dataset['relative_price'] = dataset['observed_price']/dataset['spot_price']
dates = dataset['date'].drop_duplicates().copy().reset_index(drop=True)
dates
# dataset = dataset[(dataset['w']=='put')&(dataset['barrier_type_name']=='DownOut')].reset_index(drop=True)

0      2022-02-16
1      2022-02-17
2      2022-02-18
3      2022-02-22
4      2022-02-23
          ...    
664    2024-10-09
665    2024-10-10
666    2024-10-11
667    2024-10-14
668    2024-10-15
Name: date, Length: 669, dtype: object

In [5]:
dates = pd.Series(np.sort(dataset['date'].unique()))
development_dates = dates[:len(dates)//4]
test_dates = dates[~dates.isin(development_dates)]
train_data = dataset[dataset['date'].isin(development_dates)]
test_data = dataset[dataset['date'].isin(development_dates)]
train_data

,spot_price,strike_price,barrier,days_to_maturity,updown,outin,w,barrier_type_name,rebate,dividend_rate,...,eta,v0,calculation_date,date,barrier_price,observed_price,moneyness,relative_moneyness,relative_barrier,relative_price
0,4441.75,3997.575,2220.875,60.0,Down,Out,call,DownOut,0.0,0.0,...,3.896447,6.853079e-02,2022-02-16 14:03:38.885,2022-02-16,515.613589,515.610834,444.175,0.1,0.5,0.116083
1,4441.75,3997.575,2220.875,60.0,Down,Out,put,DownOut,0.0,0.0,...,3.896447,6.853079e-02,2022-02-16 14:03:38.885,2022-02-16,37.861034,37.896135,-444.175,-0.1,0.5,0.008532
2,4441.75,3997.575,2220.875,60.0,Down,In,call,DownIn,0.0,0.0,...,3.896447,6.853079e-02,2022-02-16 14:03:38.885,2022-02-16,0.000000,0.029832,444.175,0.1,0.5,0.000007
3,4441.75,3997.575,2220.875,60.0,Down,In,put,DownIn,0.0,0.0,...,3.896447,6.853079e-02,2022-02-16 14:03:38.885,2022-02-16,7.031708,7.015353,-444.175,-0.1,0.5,0.001579
4,4441.75,3997.575,2220.875,90.0,Down,Out,call,DownOut,0.0,0.0,...,3.896447,6.853079e-02,2022-02-16 14:03:38.885,2022-02-16,546.301779,546.297068,444.175,0.1,0.5,0.122991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7002454,3586.50,3945.150,5379.750,540.0,Up,In,put,UpIn,0.0,0.0,...,8.095874,1.218996e-08,2022-10-14 16:56:27.316,2022-10-14,17.514972,17.512228,358.650,0.1,1.5,0.004883
7002455,3586.50,3945.150,5379.750,720.0,Up,Out,call,UpOut,0.0,0.0,...,8.095874,1.218996e-08,2022-10-14 16:56:27.316,2022-10-14,51.618563,51.612386,-358.650,-0.1,1.5,0.014391
7002456,3586.50,3945.150,5379.750,720.0,Up,Out,put,UpOut,0.0,0.0,...,8.095874,1.218996e-08,2022-10-14 16:56:27.316,2022-10-14,636.430811,636.419099,358.650,0.1,1.5,0.177449
7002457,3586.50,3945.150,5379.750,720.0,Up,In,call,UpIn,0.0,0.0,...,8.095874,1.218996e-08,2022-10-14 16:56:27.316,2022-10-14,560.868221,560.856698,-358.650,-0.1,1.5,0.156380


In [6]:
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.neural_network import MLPRegressor

In [7]:
numerical_scaler = StandardScaler() 

In [8]:
def make_dnn_pipeline(feature_set,numerical_scaler=numerical_scaler):
    preprocessor = ColumnTransformer([
        ('scaling',numerical_scaler,feature_set[:-2]),
        ('encoding',OneHotEncoder(),['w','barrier_type_name'])
    ])
    dnn_pipeline = make_pipeline(preprocessor,MLPRegressor(max_iter=1000,random_state=1312,solver='sgd'))
    return TransformedTargetRegressor(
        regressor=dnn_pipeline,
        transformer=Pipeline([("scaling", numerical_scaler)])
    )

In [9]:
feature_set = [
    'relative_moneyness',
    'days_to_maturity',
    'risk_free_rate',
    'dividend_rate',
    'kappa',
    'theta',
    'rho',
    'eta',
    'v0',
    'relative_barrier',
    'barrier_type_name',
    'w'
]

In [10]:
train_X = train_data[feature_set]
train_y = train_data['relative_price']

dnn = make_dnn_pipeline(feature_set,numerical_scaler)

model_fit = dnn.fit(train_X,train_y)

train_pred = model_fit.predict(train_X)

MAE = np.mean(np.abs(train_pred-train_data['relative_price']))

print('MAE:',MAE)
model_fit

E:\Python\Lib\site-packages\sklearn\preprocessing\_data.py:1020: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(self.var_), copy=False, constant_mask=constant_mask
E:\Python\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:697: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


MAE: 0.004694873594962674


TransformedTargetRegressor(regressor=Pipeline(steps=[('columntransformer',
                                                      ColumnTransformer(transformers=[('scaling',
                                                                                       StandardScaler(),
                                                                                       ['relative_moneyness',
                                                                                        'days_to_maturity',
                                                                                        'risk_free_rate',
                                                                                        'dividend_rate',
                                                                                        'kappa',
                                                                                        'theta',
                                                                                        'rho',
                                                                                        'eta',
                                                                                        'v0',
                                                                                        'relative_barrier']),
                                                                                      ('encoding',
                                                                                       OneHotEncoder(),
                                                                                       ['w',
                                                                                        'barrier_type_name'])])),
                                                     ('mlpregressor',
                                                      MLPRegressor(max_iter=1000,
                                                                   random_state=1312,
                                                                   solver='sgd'))]),
                           transformer=Pipeline(steps=[('scaling',
                                                        StandardScaler())]))

In [14]:
models_dir = os.path.join(root,ms.trained_models)

filetag = 'Oosterlee test'
train_end = time.time()
train_end_tag = str(datetime.fromtimestamp(
    train_end).strftime("%Y_%m_%d %H%M%S%f"))
file_tag = str(train_end_tag + " " + filetag)
files_dir = os.path.join(models_dir,file_tag)

def save_model():
    if Path(files_dir).exists():
        pass
    else:
        os.mkdir(files_dir)
    file_dir = os.path.join(files_dir,file_tag)
    n_calls = train_data[train_data['w']=='call'].shape[0]
    n_puts = train_data[train_data['w']=='put'].shape[0]
    train_data.to_csv(f"{file_dir} train_data.csv")
    print(f"train data saved to {file_dir}")
    test_data.to_csv(f"{file_dir} test_data.csv")
    print(f"test data saved to {file_dir}")
    joblib.dump(model_fit,str(f"{file_dir}.pkl"))
    print(f"model saved to {file_dir}")

print(f"execute the command 'save_model()' to save the following model: {file_tag}")

execute the command 'save_model()' to save the following model: 2024_11_05 184958719691 Oosterlee test


In [15]:
save_model()

train data saved to E:\OneDrive - rsbrc\git_data\machine-learning-option-pricing\trained_models_dump\trained_models\2024_11_05 184958719691 Oosterlee test\2024_11_05 184958719691 Oosterlee test
test data saved to E:\OneDrive - rsbrc\git_data\machine-learning-option-pricing\trained_models_dump\trained_models\2024_11_05 184958719691 Oosterlee test\2024_11_05 184958719691 Oosterlee test
model saved to E:\OneDrive - rsbrc\git_data\machine-learning-option-pricing\trained_models_dump\trained_models\2024_11_05 184958719691 Oosterlee test\2024_11_05 184958719691 Oosterlee test


In [ ]:
feature_set = [
    'spot_price',
    'strike_price',
    'days_to_maturity',
    'risk_free_rate',
    'dividend_rate',
    'kappa',
    'theta',
    'rho',
    'eta',
    'v0',
    'barrier',
    'barrier_type_name',
    'w'
]

In [ ]:
train_X = train_data[feature_set]

train_y = train_data['observed_price']

dnn = make_dnn_pipeline(feature_set,numerical_scaler)

model_fit = dnn.fit(train_X,train_y)
train_pred = model_fit.predict(train_X)
print('MAE:',np.mean(np.abs(train_pred-train_data['observed_price'])))
model_fit

In [ ]:
from convsklearn import barrier_trainer
help(barrier_trainer)

In [ ]:
preprocessed = barrier_trainer.get_train_test_arrays(train_data, test_data)
preprocessed.keys()

In [ ]:
old_dnn = barrier_trainer.run_dnn(preprocessor,train_X,train_y)
train_pred = old_dnn.predict(train_X)
print('MAE:',np.mean(np.abs(train_pred-train_data['spot_price'])))